In [23]:
import torch
import torch.nn as nn
from torch import Tensor
from transformers import GPTNeoXForCausalLM, AutoTokenizer, AutoModelForCausalLM
from jaxtyping import Float, Int
from typing import List, Optional, Tuple, Dict
import sys
from functools import partial
from tqdm import tqdm
import json
import seaborn as sns
import pandas as pd
import multiprocessing
import pickle

from request_patching import request_patch_one_pair, create_patch_request_dict, baseline_completion, baseline_completion_plus
from models import get_model_from_name

import warnings
warnings.filterwarnings("ignore")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('device =', device)
torch.set_grad_enabled(False)

if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"Device {i}: {torch.cuda.get_device_name(i)}")
    print(f"Current CUDA device: {torch.cuda.current_device()}")

else:
    print("CUDA is not available. Listing CPUs instead.")
    print(multiprocessing.cpu_count())

device = cpu
CUDA is not available. Listing CPUs instead.
48


In [31]:
dict = create_patch_request_dict('pythia-1b',
                                 dataset='dialogs')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Number of stories: 3


### temp

In [10]:
model, tokenizer = get_model_from_name('pythia-2.8b')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [30]:
dict

{'pair_0': {'context_1': "Bob: Hey Alice, how was your day?\nAlice: Oh, Bob, it was fantastic! I'm still riding the high from it all!\nBob: That sounds wonderful! What happened to make it so great?\nAlice: Well, for starters, the weather was absolutely perfect. Not too hot, not too cold, just the kind of day you want to bottle up and keep forever. And then, on my way to work, my favorite song played on the radio. It felt like a sign that the day was going to be amazing.\nBob: I love when that happens! It's like the universe is giving you a personal thumbs up. What else happened?\nAlice: Oh, it gets better. When I got to work, I found out that I received the promotion I've been hoping for. It was such a surprise! I've worked so hard for this, Bob, and it finally paid off.\nBob: Alice, that's incredible news! Congratulations on the promotion! You totally deserve it.\nAlice: Thank you so much! And there's more. During lunch, I went out with a few colleagues to celebrate, and we ended up h